# 運用系処理

## インデックス一覧
インデックス一覧を表示します。  
既定ではrawdata、metadata、shots、shots-analyzedインデックスのみ表示します。全てのインデックスを表示する場合、  
show_all_index=Trueを指定してください。

In [10]:
from elastic_manager import ElasticManager

ElasticManager.show_indices(show_all_index=False)

,index,docs.count,store.size
0,rawdata-20201113-1,361818,27741


## インデックス削除
DELETE_INDEXに削除するインデックス名を入力します。ワイルドカード(\*)が使用可能です。  
例）rawdata-20201201\*  
⇒ 2020/12/01の生データインデックスがすべて削除されます。

In [11]:
DELETE_INDEX = 'rawdata-20201113-*'

In [12]:
from elastic_manager import ElasticManager

ElasticManager.delete_index(DELETE_INDEX)

rawdata-20201113-*を削除しました。


## データ削除（連番指定）
連番指定でデータを削除します。以下のパラメータを指定してください。

* TARGET_INDEX：データ削除対象のインデックス名  
* START_SEQ_NUM：開始番号  
* END_SEQ_NUM：終了番号

In [2]:
TARGET_INDEX = 'rawdata-20201112-1'
START_SEQ_NUM = 0
END_SEQ_NUM = 10

In [3]:
from elastic_manager import ElasticManager

ElasticManager.delete_data_by_seq_num(index=TARGET_INDEX, start=START_SEQ_NUM, end=END_SEQ_NUM)

データを11件削除しました。


## データ削除（ショット番号指定）
ショット番号指定でデータを削除します。以下のパラメータを指定してください。  

* TARGET_INDEX：データ削除対象のインデックス名  
* SHOT_NUMBER：ショット番号

In [ ]:
TARGET_INDEX = 'shots-20201112'
SHOT_NUMBER = 1

In [ ]:
from elastic_manager import ElasticManager

ElasticManager.delete_data_by_shot_num(index=TARGET_INDEX, shot_number=SHOT_NUMBER)